In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [ ]:
import tkinter as tk
from tkinter import filedialog

In [ ]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

In [ ]:
df2018 = pd.read_excel(file_path)

In [ ]:
df2019 = pd.read_excel(file_path)

In [ ]:
df = pd.concat([df2018, df2019], axis=0)

In [ ]:
df.columns = ['年度', '類別', '內徑', '成品代號', '規格', '花纹', '公司名稱', '一月', '二月', '三月',
       '四月', '五月', '六月', '七月', '八月', '九月', '十月', '十一月', '十二月', '当月', 'TOTAL']

In [ ]:
df = df[['年度', '類別', '內徑', '成品代號', '規格', '花纹', '公司名稱', '一月', '二月', '三月',
       '四月', '五月', '六月', '七月', '八月', '九月', '十月', '十一月', '十二月']]

In [ ]:
df = df.groupby(['年度', '類別', '內徑', '成品代號', '規格', '花纹', '公司名稱']).sum().stack().reset_index()

In [ ]:
df.columns = ['年度', '類別', '內徑', '成品代號', '規格', '花纹', '公司名稱', '月份', "數量"]

In [ ]:
df["公司名稱"] = df["公司名稱"].astype("str")

In [ ]:
customer = df["公司名稱"].unique().tolist()

In [ ]:
cart_order = []
for i in customer:
    order_list = df[df["公司名稱"] == i]["規格"].values
    cart_order.append(order_list)

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

In [ ]:
te = TransactionEncoder()
te.ary = te.fit(cart_order).transform(cart_order)

In [ ]:
test_frame = pd.DataFrame(te.ary, columns=te.columns_)

In [ ]:
from mlxtend.frequent_patterns import apriori
frequent_items = apriori(test_frame, min_support=0.02, use_colnames=True)

In [ ]:
frequent_items["length"] = frequent_items["itemsets"].apply(lambda x: len(x))
frequent_items

In [ ]:
import plotly.express as px

In [ ]:
categories = df["月份"].unique().tolist()

In [ ]:
fig = px.treemap(df, path=["年度", "公司名稱", "花纹", "內徑", "規格"], values="數量",
                title="2018/2019出貨樹狀圖")


In [ ]:
line_chart = df.groupby(["年度", "公司名稱", "月份"])[["數量"]].sum().reset_index()

In [ ]:
line_chart["月份"] = pd.Categorical(line_chart["月份"], ordered=True, categories=categories)

In [ ]:
line_chart["年度"]  = line_chart["年度"].astype("str")

In [ ]:
line_chart = line_chart.sort_values(by="月份")

In [ ]:
fig2 = px.bar(line_chart, x="月份", y="數量", color="公司名稱", category_orders={"月份": categories}, facet_col="年度")

In [ ]:
with open("冬季胎分析.html", "a") as f:
    f.write(fig.to_html(full_html=False, include_plotlyjs="cdn"))
    f.write(fig2.to_html(full_html=False, include_plotlyjs="cdn"))
    f.close()